In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import os
import asf_search as asf
from collections import defaultdict
import datetime
import pickle
from tqdm import tqdm
from functools import reduce
import itertools
from shapely import Polygon, Point
import operator
import lxml
from pykml.factory import KML_ElementMaker as KML

In [2]:
f = xr.open_dataset('./data/NO_TS_MO_6200150.nc')

#df = valid_data_extraction(f, ('2014-06-20', '2022-12-31'), ['WSPD'])#['WSPD', 'VAVH'])

f

<xarray.Dataset>
Dimensions:      (TIME: 53169, DEPTH: 1, LATITUDE: 53169, POSITION: 53169,
                  LONGITUDE: 53169)
Coordinates:
  * TIME         (TIME) datetime64[ns] 2014-06-20T10:00:00 ... 2022-10-25T18:...
    DEPH         (TIME, DEPTH) float32 ...
  * LATITUDE     (LATITUDE) float32 53.6 53.6 53.6 53.6 ... 53.6 53.6 53.6 53.6
  * LONGITUDE    (LONGITUDE) float32 0.7 0.7 0.7 0.7 0.7 ... 0.7 0.7 0.7 0.7 0.7
Dimensions without coordinates: DEPTH, POSITION
Data variables: (12/19)
    TIME_QC      (TIME) float32 ...
    DEPH_QC      (TIME, DEPTH) float32 ...
    DEWT         (TIME, DEPTH) float64 ...
    DEWT_QC      (TIME, DEPTH) float32 ...
    ATMS         (TIME, DEPTH) float64 ...
    ATMS_QC      (TIME, DEPTH) float32 ...
    ...           ...
    VTZA_DM      (TIME, DEPTH) object ...
    WSPD         (TIME, DEPTH) float64 ...
    WSPD_QC      (TIME, DEPTH) float32 ...
    WDIR         (TIME, DEPTH) float64 ...
    WDIR_QC      (TIME, DEPTH) float32 ...
    POSITION_QC  (POSITION) float32 ...
Attributes: (12/46)
    platform_code:                  6200150
    platform_name:                  
    data_mode:                      M
    title:                          Atlantic-European North West Shelf-Ocean ...
    summary:                        Oceanographic data collated by the North ...
    naming_authority:               Copernicus Marine In Situ
    ...                             ...
    doi:                            
    pi_name:                        
    qc_manual:                      Recommendations for in-situ data Near Rea...
    date_update:                    2023-01-01T07:07:23Z
    history:                        2023-01-01T07:07:23Z : Creation
    wmo_inst_type:

#these are the files with changing coordinates (moving bouys)
"""files = [
    'BO_TS_MO_Kristineberg2.nc',
    'BO_TS_MO_Vaderoarna.nc',
    'BO_TS_MO_L9BS.nc',
    'BO_TS_MO_LysekilWR.nc',
    'BO_TS_MO_Koster.nc',
    'BO_TS_MO_Kristineberg1.nc',
    'BO_TS_MO_Havstensfjord.nc',
    'BO_TS_MO_DrogdenCU.nc',
    'BO_TS_MO_Tangesund.nc',
    'BO_TS_MO_BrofjordenWR.nc',
    'BO_TS_MO_VaderoarnaWR.nc',
    'BO_TS_MO_Kristineberg3.nc',
    'BO_TS_MO_LasoOstWR.nc',
    'BO_TS_MO_Dalbosjon.nc',
    'BO_TS_MO_LasoOst.nc',
    'BO_TS_MO_VengeanceCU.nc'
]"""

In [3]:
def valid_data_extraction(ds, time_filter, colum_names):
    #Make sure colum_names is a list
    if type(colum_names) is str:
        colum_names = [colum_names]
        
    #Add longitude, latidude and position_qc as variables indexed by time,depth as all other variables
    TIME = ds['TIME'].values
    DEPTH = ds['DEPTH'].values
    n_DEPTHS = len(DEPTH)

    dataset_columns = {
        'LONG':ds['LONGITUDE'],
        'LAT':ds['LATITUDE'],
        'POS_QC':ds['POSITION_QC'],
    }

    ds_pos = xr.Dataset(
        data_vars=
        {k:(
            ["TIME", 'DEPTH'],
            np.repeat(np.reshape(v.values, (-1,1)), n_DEPTHS, axis=1),
            v.attrs,
        )for (k,v) in dataset_columns.items()},
        coords=dict(
            TIME=TIME,
            DEPTH=DEPTH,
        )
    ).drop_vars('DEPTH')
    ds = xr.merge([ds.drop_dims(['LATITUDE', 'LONGITUDE', 'POSITION']), ds_pos])
    
    #Filter for time of interest and depth 0
    if type(time_filter) is tuple:
        ds = ds.sel(TIME=slice(time_filter[0], time_filter[1]), DEPTH=0)
    else:
        ds = ds.sel(TIME=time_filter, DEPTH=0)
        
    
    #Filter only avalible columns
    colum_names = list(set(colum_names).intersection(list(ds.data_vars)))
    colum_names_qc = [c + '_QC' for c in colum_names]
    
    #Add fixed columns
    colum_names.extend(['LONG', 'LAT'])
    colum_names_qc.extend(['TIME_QC', 'POS_QC'])
    
    #Filter for columns of interest
    ds = ds[colum_names + colum_names_qc]
    
    df = ds.to_dataframe()
    QC_good = [1.0, 7.0]
    
    #Filter for good data only
    filter_qc = [df[c_qc].isin(QC_good) for c_qc in colum_names_qc]
    #Element-wise AND the filters
    filter_qc = reduce(operator.and_, filter_qc)
    df = df[filter_qc][colum_names]
    
    
    return df

#Returns the (longitude,latitude) pair if all the cordinates are the same in respective dimension
def get_long_lat(dataset):  
    long_arr = dataset.coords['LONGITUDE'].values
    long = long_arr[0]
    if (not np.all(long_arr == long)):
        raise ValueError("Not all Longitude coordinates are equal")
        
    lat_arr = dataset.coords['LATITUDE'].values
    lat = lat_arr[0]
    if (not np.all(lat_arr == lat)):
        raise ValueError("Not all Latitude coordinates are equal")
        
    return long,lat

In [4]:
def search_file(file, data_dir, start_date, end_date, variables, max_time_diff_s, result_df):
    #Conditionally create the result dataframe
    if result_df is None:
        result_df = pd.DataFrame({c: pd.Series(dtype=t) for c, t in {
            'bouy_file_name':str,
            'bouy_longitude':float,
            'bouy_latitude':float,
            'bouy_time':np.dtype('<M8[ns]'), #np.datetime64
            'bouy_variable_name':str,
            'bouy_variable_value':float,
            'sar_url':str,
            'sar_mode':str,
            'sar_polarization':str,
            'sar_platform':str,
            'sar_start_time':np.dtype('<M8[ns]'), #np.datetime64
            'sar_stop_time':np.dtype('<M8[ns]'), #np.datetime64
            'sar_coordinates':object, #List
        }.items()})

    #Load the data from the file
    file_path = os.path.join(data_dir, file)
    xar = xr.open_dataset(file_path)# , engine='scipy')
    xar_vars = list(xar.data_vars)

    #Clear any eventual prior asf search result
    asf_results = None
    
    #Filter for variables that exist in the data
    common_variables = set(variables).intersection(xar_vars)
    
    if len(common_variables) == 0:
        print(file + " does not have any of the variables " + str(variables))
        return result_df
    
    for var in common_variables:
        #extract dataframe of the correct timeinterval and variable 
        df = valid_data_extraction(xar, (start_date, end_date), var)
        
        if df.empty:
            print(file + " does not have any valid data for this timeperiod")
            continue
            
        #Only search once (assume similar results in regards to min and max longitude and latitude for each variable)
        if asf_results is None:
            #extract long and latitude
            long_min, lat_min = df[['LONG', 'LAT']].min()
            long_max, lat_max = df[['LONG', 'LAT']].max()

            #Create geographical search restriction
            coord_points = list(itertools.product(set([long_min, long_max]), set([lat_min, lat_max])))

            if len(coord_points) == 1:
                geo_prefix = 'POINT('
                geo_suffix = ')'
            elif len(coord_points) == 2:
                geo_prefix = 'LINESTRING('
                geo_suffix = ')'
            elif len(coord_points) == 4:
                #Double parenthesis was given from asf website export search as python function
                geo_prefix = 'POLYGON(('
                geo_suffix = '))'
                #Untangle the polygon itersection
                coord_points[2:4]=reversed(coord_points[2:4])
                #Form closed polygon by adding adding fist point as last
                coord_points.append(coord_points[0])

            geo_limit = ','.join([str(long) + ' ' + str(lat) for long,lat in coord_points])
            geo_limit = geo_prefix + geo_limit + geo_suffix
            
            print(geo_limit)

            #Search asf
            options = {
                'intersectsWith': geo_limit,
                'platform': 'SENTINEL-1',
                'instrument': 'C-SAR',
                'start': start_date,
                'end': end_date,
                'processingLevel': [
                    'GRD_HD',
                    'GRD_MD',
                    'GRD_MS',
                    'GRD_HS'
                ],
                'beamSwath': [
                    'IW',
                    'EW'
                ],
            }

            asf_results = asf.search(**options)
            print(len(asf_results))
        
        for asf_result in asf_results:
            #Find mean time of image generation
            asf_result_start = datetime.datetime.fromisoformat(asf_result.properties['startTime'][:-5])
            asf_result_stop = datetime.datetime.fromisoformat(asf_result.properties['stopTime'][:-5])
            asf_result_mean_time = asf_result_start + (asf_result_stop-asf_result_start)/2
        
            #Check the shape type
            if asf_result.geometry['type'] != 'Polygon':
                raise ValueError('The shape of the search result is not Polygon but instead: ' + asf_result.geometry['type'])
            
            #Filter for points that lie within the image
            if geo_prefix != 'POINT(':
                polygon = Polygon(asf_result.geometry['coordinates'][0])
                df_dist_filter =  df.apply(lambda row: Point([row['LONG'], row['LAT']]).distance(polygon), axis=1) <= 0.0001
                df_close = df[df_dist_filter]
                
                #Skip is we have no overlaping points
                if df_close.empty:
                    #print("No geographical overlap")
                    continue
                    
            #No geographical overlap check needed (point search)
            else:
                df_close = df
            
            #Find closest entry in time
            try :
                closest_data_row = df_close.iloc[df_close.index.get_indexer([asf_result_mean_time], method='nearest')] #only one row of data
            except Exception as e:
                print(e)
                print(df_close)
                print(asf_result_mean_time)
                print(closest_data_row)
                raise Exception()

            max_time_diff = datetime.timedelta(seconds=max_time_diff_s)
            
            #Check if it is close enough to be considered overlapping
            if abs(closest_data_row.index[0] - asf_result_mean_time) <= max_time_diff:
                #Save result in the dataframe
                result_df.loc[len(result_df.index)] = [
                    file, #'bouy_file_name':str,
                    closest_data_row['LONG'][0], #'bouy_longitude':float,
                    closest_data_row['LAT'][0], #'bouy_latitude':float,
                    closest_data_row.index[0], #'bouy_time':np.dtype('<M8[ns]'), #np.datetime64
                    var, #'bouy_variable_name':str,
                    closest_data_row[var][0], #'bouy_variable_value':float,
                    asf_result.properties['url'], #'sar_url':str,
                    asf_result.properties['beamModeType'], #'sar_mode':str,
                    asf_result.properties['polarization'].split('+'), #'sar_polarization':str,
                    asf_result.properties['platform'], #'sar_platform':str,
                    asf_result_start, #'sar_start_time':np.dtype('<M8[ns]'), #np.datetime64
                    asf_result_stop, #'sar_stop_time':np.dtype('<M8[ns]'), #np.datetime64
                    asf_result.geometry['coordinates'][0], #'sar_coordinates':list,
                ]
                
            else:
                #print('No time overlap')
                pass
            
        
    return result_df

In [5]:
data_dir = './data/'
file_prefixes = ['NO_TS_MO', 'BO_TS_MO']
start_date = '2021-01-01'
end_date = '2021-12-31'
variables = ['VHM0', 'VAVH', 'WSPD']#['VHM0', 'VAVH', 'WSPD']
max_time_diff_s = 15*60

write_folder = './15min'
result_df_fn = 'result_df'
kml_pinmap_fn = 'kml_pinmap'

#Filtered bouy to not include, manually filtered, removing all bouys rougly < 3km to shore
file_filter = [
    'BO_TS_MO_Dalbosjon.nc',
    'BO_TS_MO_Koster.nc',
    'BO_TS_MO_Vaderobod.nc',
    'BO_TS_MO_Vaderoarna1.nc',
    'BO_TS_MO_Borno.nc',
    'BO_TS_MO_Havstensfjord.nc',
    'BO_TS_MO_HavstensfjordBS.nc',
    'BO_TS_MO_Kristineberg1.nc',
    'BO_TS_MO_Kristineberg2.nc',
    'BO_TS_MO_Tangesund.nc',
    'NO_TS_MO_BuoyHoernumTief1.nc',
    'NO_TS_MO_Hoernum.nc',
    'NO_TS_MO_HelgolandWR.nc',
    'NO_TS_MO_CuxhavenFerryBox.nc',
    'NO_TS_MO_PileSpiekeroog.nc',
    'NO_TS_MO_MeetboeiRZGN1.nc',
    'NO_TS_MO_Borkum.nc',
    'NO_TS_MO_MeetboeiUHW1.nc',
    'NO_TS_MO_Uithuizerwad2.nc',
    'NO_TS_MO_Uithuizerwad3.nc',
    'NO_TS_MO_SchiermonnikoogWaggen.nc',
    'NO_TS_MO_Wierumerwad3.nc',
    'NO_TS_MO_Wierumerwad2.nc',
    'NO_TS_MO_Amelander31.nc',
    'NO_TS_MO_Amelander32.nc',
    'NO_TS_MO_Amelander41.nc',
    'NO_TS_MO_Amelander42.nc',
    'NO_TS_MO_Amelander51.nc',
    'NO_TS_MO_Amelander52.nc',
    'NO_TS_MO_Amelander61.nc',
    'NO_TS_MO_Amelander62.nc',
    'NO_TS_MO_StortemelkOost.nc',
    'NO_TS_MO_StortemelkBoei.nc',
    'NO_TS_MO_Pannengat1.nc',
    'NO_TS_MO_Breesem1.nc',
    'NO_TS_MO_KornwerderzandBuiten.nc',
    'NO_TS_MO_Breezanddijkbuiten1.nc',
    'NO_TS_MO_Malzwin1.nc',
    'NO_TS_MO_MeetboeiPBW1.nc',
    'NO_TS_MO_DenHelder.nc',
    'NO_TS_MO_Ijmuiden.nc',
    'NO_TS_MO_IJmondstroompaal.nc',
    'NO_TS_MO_IJgeulstroompaal1.nc',
    'NO_TS_MO_IJmondstroompaal2.nc',
    'NO_TS_MO_MaeslantkeringZeezijdeNoordMeetpaal.nc',
    'NO_TS_MO_MaeslantkeringZeezijdeZuidMeetpaal.nc',
    'NO_TS_MO_HoekVanHolland.nc',
    'NO_TS_MO_HoekVanHollandNAP.nc',
    'NO_TS_MO_zdv8-2d-MMND.nc',
    'NO_TS_MO_Brouwershavensegat8.nc',
    'NO_TS_MO_Oosterschelde4.nc',
    'NO_TS_MO_Roompotsluis.nc',
    'NO_TS_MO_KeetenBoei.nc',
    'NO_TS_MO_MarollegatMeetpaal.nc',
    'NO_TS_MO_Hansweert.nc',
    'NO_TS_MO_HansweertPunt2.nc',
    'NO_TS_MO_Bath.nc',
    'NO_TS_MO_OverloopVanValkenisse.nc',
    'NO_TS_MO_PasVanTerneuzenBoei.nc',
    'NO_TS_MO_Hoofdplaat.nc',
    'NO_TS_MO_HoofdplaatMeetpaal2.nc',
    'NO_TS_MO_HonteSloehaven.nc',
    'NO_TS_MO_ZwinBuoy.nc',
    'NO_TS_MO_ZeebruggeDam.nc',
    'NO_TS_MO_ZeebruggePortBuoy.nc',
    'NO_TS_MO_ZeebruggeWeatherStation.nc',
    'NO_TS_MO_Blankenberge.nc',
    'NO_TS_MO_OostendEasternPalisade.nc',
    'NO_TS_MO_OostendBuoy.nc',
    'NO_TS_MO_OostendWeather.nc',
    'NO_TS_MO_Raversijde.nc',
    'NO_TS_MO_NieuwpoortWind.nc',
    'NO_TS_MO_Roscoff.nc',
    'NO_TS_MO_Neustadt.nc',
    'NO_TS_MO_Kuehlungsborn.nc',
    'NO_TS_MO_BuoyHoernumTief1.nc',
    'NO_TS_MO_BuoyBunkerHill1.nc',
    'NO_TS_MO_HirtshalsWR.nc',
    'NO_TS_MO_HanstholmWR.nc',
    'NO_TS_MO_BuoyBunkerHill1.nc',
    'NO_TS_MO_BuoyHoernumTief1.nc',
    'NO_TS_MO_CadzandBoei.nc',
    'NO_TS_MO_BolVanHeist.nc',
    'NO_TS_MO_A2.nc',
    'NO_TS_MO_NieuwpoortBuoy.nc',
    'NO_TS_MO_6200303.nc',
    'BO_TS_MO_Varberg.nc',
    'BO_TS_MO_Svinbadan,nc',
    'BO_TS_MO_GreatBeltBridgeEastCU.nc',
]

#Variables where the results is saved, automatically created in search_file
result_df = None

def search_file_fixed_params(file, result_df):
    return search_file(file, data_dir, start_date, end_date, variables, max_time_diff_s, result_df)

files = [f for f in os.listdir(data_dir) if any([f.startswith(fp) for fp in file_prefixes if f not in file_filter])]

print(files)

for file in tqdm(files):
    result_df = search_file_fixed_params(file, result_df)

#Conditionally creates the folder for the result
os.makedirs(write_folder, exist_ok=True)

with open(os.path.join(write_folder, result_df_fn),'wb') as f_w:
    pickle.dump(result_df,f_w)


['BO_TS_MO_Svinbadan.nc', 'NO_TS_MO_IJgeul5.nc', 'NO_TS_MO_OostendNorth.nc', 'NO_TS_MO_6300113.nc', 'NO_TS_MO_DarsserSCU.nc', 'NO_TS_MO_AWG.nc', 'NO_TS_MO_6200304.nc', 'BO_TS_MO_VengeanceTS.nc', 'BO_TS_MO_Oskarsgrundet2.nc', 'BO_TS_MO_Vaderoarna.nc', 'BO_TS_MO_L9BS.nc', 'NO_TS_MO_MeetboeiWEO1.nc', 'NO_TS_MO_6200150.nc', 'BO_TS_MO_LysekilWR.nc', 'NO_TS_MO_ElbeWR.nc', 'NO_TS_MO_6200050.nc', 'NO_TS_MO_6200029.nc', 'NO_TS_MO_Haringvliet10.nc', 'NO_TS_MO_6300105.nc', 'NO_TS_MO_Kwintebank.nc', 'NO_TS_MO_SchiermonnikoogWestgat.nc', 'NO_TS_MO_Fjaltring.nc', 'BO_TS_MO_Laso.nc', 'NO_TS_MO_Q1.nc', 'BO_TS_MO_DrogdenTS.nc', 'NO_TS_MO_6200027.nc', 'NO_TS_MO_6200081.nc', 'NO_TS_MO_LichteilandGoeree1.nc', 'NO_TS_MO_Sleipner-A.nc', 'NO_TS_MO_IJmuidenMunitiestort2.nc', 'NO_TS_MO_Gullfaks-C.nc', 'NO_TS_MO_6200114.nc', 'NO_TS_MO_6300117.nc', 'NO_TS_MO_Butendiek.nc', 'NO_TS_MO_6200165.nc', 'NO_TS_MO_ScheurWest.nc', 'NO_TS_MO_TWEms.nc', 'NO_TS_MO_K13a2.nc', 'NO_TS_MO_DomburgerRassen.nc', 'NO_TS_MO_FehmarnBe

  1%|██                                                                                                                                                                            | 2/165 [00:00<00:11, 13.82it/s]

BO_TS_MO_Svinbadan.nc does not have any valid data for this timeperiod
NO_TS_MO_IJgeul5.nc does not have any valid data for this timeperiod
NO_TS_MO_IJgeul5.nc does not have any valid data for this timeperiod
POINT(2.882999897003174 51.35200119018555)



  1%|██                                                                                                                                                                            | 2/165 [00:19<00:11, 13.82it/s]

233


...


 98%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊   | 162/165 [36:57<00:44, 14.93s/it]

BO_TS_MO_Flinten7BS.nc does not have any of the variables ['VHM0', 'VAVH', 'WSPD']
POINT(2.227299928665161 59.57419967651367)
294



 99%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉ | 164/165 [37:18<00:13, 13.54s/it]

NO_TS_MO_Heimdal.nc does not have any valid data for this timeperiod
POINT(3.7369439601898193 52.0099983215332)
176


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 165/165 [37:34<00:00, 13.67s/it]


In [6]:
result_df

,bouy_file_name,bouy_longitude,bouy_latitude,bouy_time,bouy_variable_name,bouy_variable_value,sar_url,sar_mode,sar_polarization,sar_platform,sar_start_time,sar_stop_time,sar_coordinates
0,NO_TS_MO_OostendNorth.nc,2.883000,51.352001,2021-12-28 06:00:00,VHM0,0.61,https://datapool.asf.alaska.edu/GRD_HD/SA/S1A_...,IW,"[VV, VH]",Sentinel-1A,2021-12-28 06:06:47,2021-12-28 06:07:12,"[[3.499949, 51.225842], [4.00008, 52.718201], ..."
1,NO_TS_MO_OostendNorth.nc,2.883000,51.352001,2021-12-26 17:30:00,VHM0,0.60,https://datapool.asf.alaska.edu/GRD_HD/SA/S1A_...,IW,"[VV, VH]",Sentinel-1A,2021-12-26 17:33:26,2021-12-26 17:33:51,"[[1.552611, 52.441559], [2.048772, 50.948856],..."
2,NO_TS_MO_OostendNorth.nc,2.883000,51.352001,2021-12-23 06:00:00,VHM0,0.74,https://datapool.asf.alaska.edu/GRD_HD/SA/S1A_...,IW,"[VV, VH]",Sentinel-1A,2021-12-23 05:58:37,2021-12-23 05:59:02,"[[5.474199, 51.003326], [5.971075, 52.49585], ..."
3,NO_TS_MO_OostendNorth.nc,2.883000,51.352001,2021-12-22 06:00:00,VHM0,0.34,https://datapool.asf.alaska.edu/GRD_HD/SB/S1B_...,IW,"[VV, VH]",Sentinel-1B,2021-12-22 06:06:25,2021-12-22 06:06:50,"[[3.138019, 50.070847], [3.624137, 51.564003],..."
4,NO_TS_MO_OostendNorth.nc,2.883000,51.352001,2021-12-20 17:30:00,VHM0,0.72,https://datapool.asf.alaska.edu/GRD_HD/SB/S1B_...,IW,"[VV, VH]",Sentinel-1B,2021-12-20 17:32:36,2021-12-20 17:33:01,"[[1.748823, 51.907574], [2.237964, 50.414551],..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20742,NO_TS_MO_EurogeulE13.nc,3.736944,52.009998,2021-01-09 06:10:00,VAVH,0.88,https://datapool.asf.alaska.edu/GRD_HD/SA/S1A_...,IW,"[VV, VH]",Sentinel-1A,2021-01-09 05:58:31,2021-01-09 05:58:56,"[[5.474749, 51.003311], [5.97163, 52.495819], ..."
20743,NO_TS_MO_EurogeulE13.nc,3.736944,52.009998,2021-01-08 06:10:00,VAVH,1.39,https://datapool.asf.alaska.edu/GRD_HD/SB/S1B_...,IW,"[VV, VH]",Sentinel-1B,2021-01-08 06:05:54,2021-01-08 06:06:19,"[[3.625535, 51.564022], [4.13135, 53.056099], ..."
20744,NO_TS_MO_EurogeulE13.nc,3.736944,52.009998,2021-01-06 17:30:00,VAVH,0.97,https://datapool.asf.alaska.edu/GRD_HD/SB/S1B_...,IW,"[VV, VH]",Sentinel-1B,2021-01-06 17:32:29,2021-01-06 17:32:54,"[[1.747713, 51.907463], [2.236964, 50.414387],..."
20745,NO_TS_MO_EurogeulE13.nc,3.736944,52.009998,2021-01-03 06:00:00,VAVH,0.74,https://datapool.asf.alaska.edu/GRD_HD/SB/S1B_...,IW,"[VV, VH]",Sentinel-1B,2021-01-03 05:58:01,2021-01-03 05:58:26,"[[5.282145, 50.341362], [5.773809, 51.834118],..."


In [9]:
#Save KML pin maps, one for each variable

#Total number of datapoints per variable type
var_total = result_df['bouy_variable_name'].value_counts()
 
#pandas series for count of unique variable name, file name pairs
var_file_count = list(result_df.value_counts(subset=['bouy_variable_name', 'bouy_file_name']).items())

#dataframe for coordinates of unique variable name, file name pairs
var_file_coord = (result_df.groupby(['bouy_variable_name', 'bouy_file_name'])[['bouy_longitude', 'bouy_latitude']].first())


#Create separate kml maps for each variable name
KML_fldrs = {}
for var_name in np.unique(result_df['bouy_variable_name']):
    KML_fldrs[var_name] = KML.Folder(
        KML.name(var_name + " " + write_folder[write_folder.find('/')+1:]),
        KML.description('Total datapoints: ' + str(var_total[var_name])),
    )

#iterate over all unique variable name, file name pairs
for ((var_name, file_name), count) in var_file_count:
    #extract longitude and latitude
    long, lat = var_file_coord.loc[(var_name, file_name)][['bouy_longitude', 'bouy_latitude']]
    
    #Create the pin
    pin = KML.Placemark(
        KML.name(str(count)),
        KML.description(file_name),
        KML.Point(
            KML.coordinates(str(long) + "," + str(lat))
        )
    )
    KML_fldrs[var_name].append(pin)
    
for var_name, KML_fld in KML_fldrs.items():
    with open(os.path.join(write_folder, kml_pinmap_fn + '_' + var_name + '.kml'), 'w') as f_w:
        f_w.write(lxml.etree.tostring(KML_fld, pretty_print=True).decode())

In [20]:
#Save KML pin map if we have all the variables ['VHM0', 'VAVH', 'WSPD']
if all([v in variables for v in ['VHM0', 'VAVH', 'WSPD']]):
    #Number of  overlaps
    mult_var = result_df.groupby(['sar_url', 'bouy_file_name', 'bouy_longitude', 'bouy_latitude'])[['bouy_variable_name']].aggregate(lambda tdf: tdf.unique().tolist())
    mult_var['len'] = mult_var['bouy_variable_name'].apply(len)
    mult_var['contains_wind'] = mult_var['bouy_variable_name'].apply(lambda x: 'WSPD' in x)
    mult_var['also_VAVH'] = mult_var['bouy_variable_name'].apply(lambda x: 'VAVH' in x)
    mult_var['also_VHM0'] = mult_var['bouy_variable_name'].apply(lambda x: 'VHM0' in x)
    mult_var = mult_var[(mult_var['len'] > 1) & mult_var['contains_wind']].drop(labels='contains_wind', axis=1)

    wind_and_VAVH = mult_var[mult_var['also_VAVH']].drop(labels=['also_VAVH', 'also_VHM0'], axis=1).groupby(['bouy_file_name', 'bouy_longitude', 'bouy_latitude'])['len'].count()
    wind_and_VHM0 = mult_var[mult_var['also_VHM0']].drop(labels=['also_VAVH', 'also_VHM0'], axis=1).groupby(['bouy_file_name', 'bouy_longitude', 'bouy_latitude'])['len'].count()

    #Save overlapping wind and wave info to KML maps
    for kml_pinmap_fn, data_var in [('WSPD_and_VHM0', wind_and_VHM0), ('WSPD_and_VAVH', wind_and_VAVH)]:
        KML_fldr = KML.Folder(
            KML.name(kml_pinmap_fn + " " + write_folder[write_folder.find('/')+1:]),
            KML.description('Total datapoints: ' + str(data_var.sum())),
        )

        #iterate over all unique variable name, file name pairs
        for ((file_name, long, lat), count) in data_var.items():
            #Create the pin
            pin = KML.Placemark(
                KML.name(str(count)),
                KML.description(file_name),
                KML.Point(
                    KML.coordinates(str(long) + "," + str(lat))
                )
            )
            KML_fldr.append(pin)

        with open(os.path.join(write_folder, kml_pinmap_fn + '_' + var_name + '.kml'), 'w') as f_w:
            f_w.write(lxml.etree.tostring(KML_fldr, pretty_print=True).decode())